<a href="https://colab.research.google.com/github/bhavepurva/Movie-Recommender/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#movie recommendation system

#using rating provided by MovieLens and also use lenskit api instead of doing everything from scratch
!pip install lenskit
import lenskit.datasets as ds
import pandas as pd

!git clone https://github.com/crash-course-ai/lab4-recommender-systems.git

data = ds.MovieLens('lab4-recommender-systems/')



     |████████████████████████████████| 92kB 5.8MB/s 
     |████████████████████████████████| 133kB 14.6MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219255 sha256=18a644368f2d9bb58eb6f8ae60fc14d46ce04d29b0024473666a566360126f8d
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
Cloning into 'lab4-recommender-systems'...
remote: Enumerating objects: 25, done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 25
Unpacking objects: 100% (25/25), done.


In [6]:
#view data, note movies, ratings , tags are in separate csv files

data.ratings.head(10)

#including only those movies which got high rates by more than 20 people

avg_rating=(data.ratings).groupby(["item"]).mean()
rating_count=(data.ratings).groupby(["item"]).count()
avg_rating=avg_rating.loc[rating_count["rating"]>20]
avg_rating=avg_rating.join(data.movies["genres"],on="item")
avg_rating=avg_rating.join(data.movies["title"],on="item")
avg_rating=avg_rating.loc[avg_rating["genres"].str.contains("Horror","Thriller")]

x=avg_rating.sort_values(by="rating",ascending=False)

x.head(20)


,user,rating,timestamp,genres,title
item,,,,,
2160,357.343750,4.171875,1.215725e+09,Drama|Horror|Thriller,Rosemary's Baby (1968)
593,306.577061,4.161290,1.147081e+09,Crime|Horror|Thriller,"Silence of the Lambs, The (1991)"
1258,313.908257,4.082569,1.228276e+09,Horror,"Shining, The (1980)"
1261,342.500000,4.044118,1.157846e+09,Action|Comedy|Fantasy|Horror,Evil Dead II (Dead by Dawn) (1987)
1215,314.764706,4.039216,1.154281e+09,Action|Adventure|Comedy|Fantasy|Horror,Army of Darkness (1993)
1219,309.156627,4.036145,1.183471e+09,Crime|Horror,Psycho (1960)
93840,346.045455,4.022727,1.429735e+09,Comedy|Horror|Sci-Fi|Thriller,"Cabin in the Woods, The (2012)"
8874,337.948052,4.006494,1.302222e+09,Comedy|Horror,Shaun of the Dead (2004)
1387,335.296703,4.005495,1.149149e+09,Action|Horror,Jaws (1975)


In [7]:
#define recommender
#this finds clusters of similar taste users

from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

num_recs=30

user_user=UserUser(15,min_nbrs=3)
model=Recommender.adapt(user_user)
model.fit(data.ratings)


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB
BLAS using multiple threads - can cause oversubscription
found 2 potential runtime problems - see https://boi.st/lkpy-perf


In [8]:
#pass in our movies and see which movies get recommended

recs=model.recommend(-1,num_recs,ratings=pd.Series(x["rating"]))
joined_data=recs.join(data.movies["genres"],on="item")
joined_data=joined_data.join(data.movies["title"],on="item")
joined_data

,item,score,genres,title
0,3677,4.870114,Documentary,Baraka (1992)
1,27156,4.777382,Action|Animation|Drama|Fantasy|Sci-Fi,Neon Genesis Evangelion: The End of Evangelion...
2,5485,4.699830,Comedy|Drama|Romance,Tadpole (2002)
3,187,4.642748,Comedy,Party Girl (1995)
4,69524,4.637074,Action|Adventure|Thriller,Raiders of the Lost Ark: The Adaptation (1989)
5,1198,4.636375,Action|Adventure,Raiders of the Lost Ark (Indiana Jones and the...
6,665,4.616473,Comedy|Drama|War,Underground (1995)
7,177593,4.613884,Crime|Drama,"Three Billboards Outside Ebbing, Missouri (2017)"
8,2936,4.612587,Adventure|Comedy|Romance,Sullivan's Travels (1941)
9,7121,4.603791,Comedy|Romance,Adam's Rib (1949)
